<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/PH_SentenceTransfomer_NounPhrases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare whether you are on Colab or local
colab = True

In [2]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')


  #install required packages
  #!pip install duckdb -q
  !pip install -U sentence-transformers -q

Mounted at /content/drive
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 4.4 MB 41.4 MB/s 
     |████████████████████████████████| 1.2 MB 72.2 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
     |████████████████████████████████| 6.6 MB 53.8 MB/s 


In [3]:
#set path to data in Google Drive
data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"
data_path_group = data_path[:-10]+"Topic 1/Data_Team1/" # create new data path to access files created by Team1


In [4]:
# Import sentence_transformers 
from sentence_transformers import SentenceTransformer, util

In [5]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#import duckdb
#import sqlalchemy
#%load_ext sql

### Load data

#### Noun phrases

In [6]:
entities_train = pd.read_csv(data_path_group+"SynDet/SynDet_entities_train.csv", index_col=0)

In [7]:
# Filter for one article (188)
entities_sample = entities_train.loc[entities_train["article_id"]==188]
entities_sample.shape

(3227, 5)

In [8]:
entities_sample.head(10)

,article_id,sentence_id,ent_id,noun_phrases,true_ent_id
0,188,188_3753_3772,videotex,which,0
1,188,188_3753_3772,videotex,we,0
2,188,188_3753_3772,videotex,an information,0
3,188,188_3753_3772,videotex,communication resource,0
4,188,188_3753_3772,videotex,the Minitel system,0
5,188,188_3828_3848,competitive advantage,It,0
6,188,188_3828_3848,competitive advantage,any rm,0
7,188,188_3828_3848,competitive advantage,the eld,0
8,188,188_3828_3848,competitive advantage,so competitive advantages,0
9,188,188_3848_3883,government,This,0


#### Ontology (entities + synonyms)

In [9]:
ontology_train = pd.read_csv(data_path_group+"SynDet/SynDet_ontology_train.csv", index_col=0)

In [ ]:
# Check number of synonyms per entity_id
ent_sy_group = ontology_train.groupby("entity_id")[["synonym"]].nunique().sort_values(by="synonym", ascending=False)

In [ ]:
# Downsample entities with more than 100 synonyms to a 100 synoyms (but always include "main entity_id" in subsample)
#idx = ent_sy_group[ent_sy_group["synonym"]>100].index.tolist()
#
#dfs = []
#for entity in idx:
#  t1 = ontology_train[(ontology_train.entity_id==entity)&(ontology_train.synonym==entity)]
#  t2 = ontology_train[(ontology_train.entity_id==entity)&(ontology_train.synonym!=entity)].sample(99)
#  dfs.append(pd.concat([t1,t2]))
#dfs.append(ontology_train.loc[~ontology_train["entity_id"].isin(idx)]) # Append dataframe for all entities with less then 100 synonyms to list

# Combine all dataframes 
#ontology_train_ds = pd.concat(dfs).reset_index(drop=True)

In [ ]:
#Show synonyms in ontology which appear in more than 1 entity
ontology_train_ds[ontology_train_ds.synonym.duplicated(keep=False)].sort_values(by="synonym")

,entity_id,category,label,synonym
22253,Pearson's Chi square test,methodological entity,ANALYSIS_METHOD,Chi2 test for independence
21871,Pearson's chi squared test of independence bet...,methodological entity,ANALYSIS_METHOD,Chi2 test for independence
21865,Pearson's chi squared test of independence bet...,methodological entity,ANALYSIS_METHOD,Chi2 test for independences
22248,Pearson's Chi square test,methodological entity,ANALYSIS_METHOD,Chi2 test for independences
26033,systems operations,domain specific entity,TOPIC,EDP operation
...,...,...,...,...
25233,information technology operation,domain specific entity,TOPIC,tool operations
26015,use case modeling,domain specific entity,TOPIC,use case diagrams
21068,use case diagram,research method,CONCEPTUAL_METHOD,use case diagrams
29501,website,domain specific entity,TECHNOLOGY,web sites


In [10]:
#List of unique entities from ontology with synonyms without downsampling (apparently there are some duplicates in there?!)
#entities_syn_unique_ds = ontology_syn_ds.synonym.unique().tolist()
ontology_train_ds_unique = ontology_train.drop_duplicates(subset="synonym")

### Load model

In [11]:
# Load a pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

### Create similartiy functions

Within the sentence tansformer package there are different functions to compute cosine similarity., after testing we chose: 
- `util.semantic_search`

In [12]:
# create function based on semantic_search
def sem_search_sent_ent(model, sentences, entities, top_k = 1):
  #Compute embeddings
  embeddings_sentences = model.encode(sentences.noun_phrases.to_list(), convert_to_tensor=True, show_progress_bar=True)
  embeddings_entities = model.encode(entities.synonym.to_list(), convert_to_tensor=True, show_progress_bar=True)

  #Compute cosine-similarities for each sentence with all entities and return top k per sentence
  cosine_scores = util.semantic_search(embeddings_sentences, embeddings_entities, top_k=top_k)

  #loop over results from semantic_search and create dataframe with sentence id and store in list
  dfs = []
  for idx, sim in enumerate(cosine_scores):
    test = pd.DataFrame(sim)
    test["sentence_id"] = sentences.iloc[idx,1]
    test["noun_phrase_txt"] = sentences.iloc[idx,3]
    test = test.rename(columns={"corpus_id":"entity_id"})
    dfs.append(test)

  # concat all dfs 
  dfs_df = pd.concat(dfs)
  dfs_df = dfs_df[["score", "sentence_id", "entity_id", "noun_phrase_txt"]] # reorder columns
  dfs_df[["main_entity_id"]] = dfs_df[["entity_id"]].applymap(lambda x:  entities.iloc[x,0]) # add "main" entity name (since we compare also with synonyms)
  dfs_df[["entity_id"]] = dfs_df[["entity_id"]].applymap(lambda x:  entities.iloc[x,3]) #replace entity_id_idx with real name of entity
  dfs_df = dfs_df.sort_values(by="score", ascending=False, ignore_index=True) # sort by score and reset index

  return dfs_df

In [13]:
%%time
sim_scores_sample = sem_search_sent_ent(model, entities_sample, ontology_train_ds_unique, top_k=1)

Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Batches:   0%|          | 0/10626 [00:00<?, ?it/s]

CPU times: user 1min 30s, sys: 3.97 s, total: 1min 34s
Wall time: 1min 38s


### Check performance of similarity scores

In order to determine the range of cosine similarity score we want to include for further processing (so we only show matches which are "fairly" similar to entities from ontology to the user) we will compare the "closest" entity to the "true" label from Rolands & Sebastians DB

In [14]:
# Set max column width to None so we can read the whole sentences
#pd.set_option('display.max_colwidth', None)

In [15]:
sim_scores_sample.head(2)

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id
0,1.000001,188_4511_4556,IT,it,IS technology
1,1.000001,188_3675_3704,IT,it,IS technology


In [16]:
# Merge sim scores on entities sample to get "true" entity id
sim_scores_sample_mapped = sim_scores_sample.merge(entities_sample
                                              [["sentence_id", "noun_phrases", "true_ent_id"]],
                                               how="left", left_on=["sentence_id", "noun_phrase_txt"], 
                                               right_on=["sentence_id", "noun_phrases"])\
                                               .drop(columns=["noun_phrases"])

In [17]:
# Define columns which should be checked for duplicates
col_dupl = sim_scores_sample_mapped.columns[1:].to_list()

#Drop duplicates
sim_scores_sample_mapped = sim_scores_sample_mapped.drop_duplicates(subset=col_dupl)

In [18]:
#Merge sim scores again on entities sample to get "true" entity id, but this time with the true entity id which was found by Roland
sim_scores_sample_mapped_f =sim_scores_sample_mapped.merge(entities_sample[["sentence_id", "ent_id"]], 
                                                           how="outer", left_on=["sentence_id", "main_entity_id"], 
                                                           right_on=["sentence_id", "ent_id"])\
                                                           .rename(columns={"ent_id":"true_ent_id_cmpl_sen"})\
                                                           .drop_duplicates()

In [19]:
# Define function to combine both columns "true_ent_id" & "true_ent_id_cmpl_sen"
def true_ent_label(x):

  if x.true_ent_id!="0":
    return x.true_ent_id
  else:
    return x.true_ent_id_cmpl_sen

In [20]:
# Compare both columns and store correct value  in column true_entity_id
sim_scores_sample_mapped_f["true_entity_id"] = sim_scores_sample_mapped_f.apply(lambda x: true_ent_label(x), 
                                                                                axis=1, result_type="expand")

#Fill all NaN Values with 0 (no match)
sim_scores_sample_mapped_f["true_entity_id"] = sim_scores_sample_mapped_f["true_entity_id"].fillna(0)

#Drop columns "true_ent_id" & "true_ent_id_cmpl_sen"
sim_scores_sample_mapped_f = sim_scores_sample_mapped_f.drop(columns=["true_ent_id", "true_ent_id_cmpl_sen"])

In [21]:
sim_scores_sample_mapped_f.head(3)

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,true_entity_id
0,1.000001,188_4511_4556,IT,it,IS technology,0
1,1.000001,188_3675_3704,IT,it,IS technology,0
2,1.000001,188_3828_3848,IT,It,IS technology,0


In [22]:
# Check wheter main_entity based on cosine similarity is identical to "true" main entity label 
sim_scores_sample_mapped_f["correct_match"] = sim_scores_sample_mapped_f.main_entity_id==sim_scores_sample_mapped_f.true_entity_id

In [23]:
sim_scores_sample_mapped_f.head(3)

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,true_entity_id,correct_match
0,1.000001,188_4511_4556,IT,it,IS technology,0,False
1,1.000001,188_3675_3704,IT,it,IS technology,0,False
2,1.000001,188_3828_3848,IT,It,IS technology,0,False


In [24]:
# Check results for one sample sentence
sim_scores_sample_mapped_f[(sim_scores_sample_mapped_f.sentence_id=="188_5984_6028")].sort_values(by="score", ascending=False)#&(sim_scores_sample_mapped.score<1.)]

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,true_entity_id,correct_match
200,1.000000,188_5984_6028,information infrastructure,information infrastructure,information technology infrastructure,information technology infrastructure,True
475,0.975544,188_5984_6028,private sector,the private sector,private sector,private sector,True
1055,0.963922,188_5984_6028,information superhighway,an information superhighway,information superhighway,information superhighway,True
2955,0.855710,188_5984_6028,public sector institutions,the public institutions,government,0,False
3095,0.834706,188_5984_6028,applications project,application,IT project,0,False
3407,0.810322,188_5984_6028,IS development,the development,systems development,0,False
3875,0.752547,188_5984_6028,information service development,service development,information services development,0,False
3877,0.752482,188_5984_6028,information and communications technologies in...,innovative communications,digital innovation,0,False
4233,0.683284,188_5984_6028,receiver operating characteristics,hardware receivers,ROC curve,0,False
4505,0.597921,188_5984_6028,wireless technologies,transmitters,mobile system,0,False


We can see that our algorithm did indeed match 3 tags correctly. We could now use the `correct_match` column to calculate some statistics on how many entities the transfomer did correctly classify (and we will do it below). However, the example above also shows that there are some entities which our algorithm classifies as very similar (e.g. the public institutions) which was not tagged in the original database. This means that the scores which we will calculate below do not reflect the true "accuracy" of our algorithm, since the original mapping of sentences is not perfect!
But we also have to note that our procedure to determine wheter the match was correct also has issues if our algorithm did not correctly predict the main entity (in the example above the sentence was also tagged as hardware, but since our algorithm matched "hardware receivers" incorrectly to "ROC curve", this faulty match does not appear in our dataframe!

In [25]:
# Create bins of similarity score to check the number of correct matches compared to the labels of the database
grouped_df = sim_scores_sample_mapped_f.groupby(pd.cut(sim_scores_sample_mapped_f.score, np.arange(0.,1.1,0.1)))[["correct_match"]]
corr_match_bins = np.round(grouped_df.sum()/grouped_df.count(),2).rename(columns={"correct_match":"pct_correct"})
corr_match_bins["no_correct"]= grouped_df.sum()
corr_match_bins["total"]= grouped_df.count()

In [26]:
corr_match_bins

,pct_correct,no_correct,total
score,,,
"(0.0, 0.1]",NaN,0,0
"(0.1, 0.2]",NaN,0,0
"(0.2, 0.3]",NaN,0,0
"(0.3, 0.4]",0.00,0,25
"(0.4, 0.5]",0.03,7,252
"(0.5, 0.6]",0.01,4,358
"(0.6, 0.7]",0.08,18,217
"(0.7, 0.8]",0.17,46,264
"(0.8, 0.9]",0.33,94,281


In [27]:
# Lets see how many noun phrases a user would have to look depending on the range we choose
for i in np.arange(0.1,1.,0.1):
  print("Number of phrases a user would need to check if we choose a similarity score range: (",\
        np.round(i,1),",1]:", \
        sim_scores_sample_mapped_f.loc[\
                                     (sim_scores_sample_mapped_f["score"]>=i)&\
                                     (sim_scores_sample_mapped_f["score"]<=0.99)].shape[0])

Number of phrases a user would need to check if we choose a similarity score range: ( 0.1 ,1]: 1502
Number of phrases a user would need to check if we choose a similarity score range: ( 0.2 ,1]: 1502
Number of phrases a user would need to check if we choose a similarity score range: ( 0.3 ,1]: 1502
Number of phrases a user would need to check if we choose a similarity score range: ( 0.4 ,1]: 1477
Number of phrases a user would need to check if we choose a similarity score range: ( 0.5 ,1]: 1225
Number of phrases a user would need to check if we choose a similarity score range: ( 0.6 ,1]: 867
Number of phrases a user would need to check if we choose a similarity score range: ( 0.7 ,1]: 650
Number of phrases a user would need to check if we choose a similarity score range: ( 0.8 ,1]: 386
Number of phrases a user would need to check if we choose a similarity score range: ( 0.9 ,1]: 105


We will now manually check some outputs close to the areas around 0.6 cos sim

In [28]:
# Lets have a look at the matches with a similarity score >0.6 to assess wheter there are some synonyms/ entities mentioned 
sim_scores_sample_mapped_f.loc[(sim_scores_sample_mapped_f["score"]>=0.6)&(sim_scores_sample_mapped_f["score"]<=0.7)].sample(10)

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,true_entity_id,correct_match
4353,0.652203,188_5614_5655,critical theory,the necessary critical mass,critical theory,0,False
4450,0.630303,188_8787_8829,smart televisions,an expensive HDTV,smart TV,0,False
4470,0.620820,188_88_121,legal context,the context,legal context,0,False
4230,0.683284,188_4077_4123,receiver operating characteristics,hardware receivers,ROC curve,0,False
4241,0.679977,188_11092_11117,book publishers,Kluwer Academic Publishers,book publishing,0,False
4145,0.695719,188_5145_5172,environmental sustainability,the environment,environmental sustainability,0,False
4337,0.654116,188_11117_11136,academic journal list,many international journals,academic journal list,0,False
4133,0.696496,188_5285_5330,local government,no state or local government,government,government,True
4486,0.610937,188_11068_11092,academic journal list,journal and book format,academic journal list,0,False
4290,0.659554,188_7792_7841,market,This market-led strategy,organizational participant,0,False
